### Setup

In [51]:
# Import main libraries
import os
import json
from bson.objectid import ObjectId
from dotenv import load_dotenv
import pandas as pd
import re
from typing import Annotated
from typing_extensions import TypedDict

# Import LLM, MongoDB etc.
from google import genai

from pymongo import MongoClient
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode

# Import LangChain components
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

# Import others



In [52]:
# load API keys from .env
load_dotenv()

True

In [53]:
# Connect to MongoDB using MONGODB_URI
mongodb_uri = os.getenv('MONGODB_URI')
client = MongoClient(mongodb_uri)
db = client['airspace']  # Create/use a database named 'airspace'
metadata_full = db['metadata_full']  # Create/use a collection named 'metadata_full'
metadata_chunk = db['metadata_chunk']  # Create/use a collection named 'metadata_chunk'

# Example: Ensure collection exists (MongoDB creates it on first insert)
print('Connected to MongoDB. airpsace DB and metadata collections are ready.')

Connected to MongoDB. airpsace DB and metadata collections are ready.


In [60]:
# List of functions

# Create a upload function to metadata from JSON string and upload it to MongoDB
def load_metadata_json_str(metadata_str: str):
    """
    Upload metadata from a JSON string to MongoDB.
    
    Args:
        metadata_str (str): JSON string containing metadata.
    """
    try:
        # Insert metadata into MongoDB
        result = metadata_full.insert_one(metadata_str)
        print(f"Metadata uploaded successfully with ID: {result.inserted_id}")
    except Exception as e:
        print(f"Error uploading metadata: {e}")


# Create a upload function to metadata from a file and upload it to MongoDB
def load_metadata_file(file_path: str): 
    """
    Upload metadata from a file to MongoDB.
    
    Args:
        file_path (str): Path to the file containing metadata in JSON format.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            metadata_str = json.load(file)
        
        # Insert metadata into MongoDB
        load_metadata_json_str(metadata_str)
    except Exception as e:
        print(f"Error uploading metadata: {e}")


# Create a function to load data from a file into a JSON string
def load_data_file(file_path: str) -> str:
    """
    Load data from a file into a pandas DataFrame.
    Return sample size of 20 rows for testing purposes.
    
    Args:
        file_path (str): Path to the data file.
        
    Returns:
        pd.DataFrame: Data loaded into a DataFrame.
    """
    try:
        df = pd.read_csv(file_path)
        n_rows = min(20, len(df))
        if n_rows == 0:
            print(f"No data found in {file_path}.")
            return ""
        df_1 = df.sample(n=n_rows, random_state=42) if len(df) > 20 else df
        df_json = df_1.to_json(orient='records', force_ascii=False)
        print(f"Data loaded successfully from {file_path}. Sample size: {n_rows} rows.")
        return df_json
    except Exception as e:
        print(f"Error loading data file: {e}")
        return ""


# Create a function to load additional information from a file to help generate metadata
def load_add_info(file_path: str) -> str:
    """
    Load additional information from a file.
    
    Args:
        file_path (str): Path to the additional info file.
        
    Returns:
        str: Additional information as a string.
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            add_info = file.read()
        print(f"Additional info loaded successfully from {file_path}.")
        return add_info
    except Exception as e:
        print(f"Error loading additional info: {e}")
        return ""

In [ ]:
# Example: Already have metadata file. Load safety_metadata.json into MongoDB
metadata_path = os.path.join('Data', 'safety_metadata.json')
load_metadata_file(metadata_path)

Metadata uploaded successfully with ID: 687b4de7b62030e1e230136b


In [ ]:
# Does not have metadata file. Example 1: Load Aircraft metadata

# Open source data file. 
source_data_path = os.path.join('Data', 'Aircraft Table.csv')

# Read CSV into DataFrame and convert to JSON
aircraft_data_json = load_data_file(source_data_path)


Data loaded successfully from Data\Aircraft Table.csv. Sample size: 12 rows.


In [ ]:
# Does not have metadata file. Example 2: Load Singapore weather metadata

# Read source data file. Another example: Singapore Weather Table
source_data_path = os.path.join('Data', 'singapore_weather_data.csv')
# Read CSV into DataFrame and convert to JSON
sgweather_data_json = load_data_file(source_data_path)


# Read additional info from weather_data_metadata.txt
add_info_path = os.path.join('Data', 'weather_data_metadata.txt')
weather_add_info = load_add_info(add_info_path)


Data loaded successfully from Data\singapore_weather_data.csv. Sample size: 20 rows.
Additional info loaded successfully from Data\weather_data_metadata.txt.


### Build a Metadata Generator Agent
The Metadata Generator Agent will analyze an uploaded data source and creates metadata using LLM.
In general, the temperature has to be low and the max_output_tokens should be the following:
| **Metadata Detail Level**                                     | **Estimated Tokens** |
| ------------------------------------------------------------- | -------------------- |
| Basic (table + column names/types)                            | 200–400              |
| Moderate (adds descriptions, constraints)                     | 400–800              |
| Detailed (adds query hints, full descriptions, relationships) | 800–1,500+           |
| Complex (multiple tables/relationships)                       | 1,500–3,000+         |

In [43]:
# Set OpenAI model configuration
MODEL_NAME = "gpt-4o"
TEMPERATURE = 0.2
MAX_TOKENS = 2000

# Initialize OpenAI client
llm = ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE, max_tokens=MAX_TOKENS)   

In [61]:
# Metadata generation system prompt template for generating metadata from a dataset.
# Escape curly braces for .format(), except for the data sample placeholder
metadata_generator_system_prompt = """
You are a metadata generation assistant. When provided with a sample dataset (e.g., CSV or relational table), your task is to analyze the data and generate metadata in the following structured JSON format.

### Instructions:
- Infer the primary key by identifying the column(s) that have unique values across all rows (i.e., 100% uniqueness). But fields such as free text and dates may not be suitable for primary keys.
- Detect foreign key relationships based on column names that suggest references (e.g., ending in `_id`, matching a primary key in the same or another table).
- Generate detailed metadata for each column.
- Follow the JSON structure below. Do not include explanations or extra text.
- If there are no suitable examples for a field, use an empty list as appropriate.

### Output JSON structure:

{{
  "database": "<name_of_database>",
  "table": "<name_of_table>",
  "description": "<brief_description_of_table_purpose>",
  "constraints": {{
    "primary_key": ["<column_name1>", "<column_name2>"],
    "foreign_keys": [
      {{
        "column": "<column_name>",
        "references_table": "<referenced_table_name>",
        "references_column": "<referenced_column_name>"
      }}
    ]
  }},
  "columns": [
    {{
      "name": "<column_name>",
      "type": "<data_type>",
      "description": "<meaning_or_purpose_of_column>",
      "examples: ["<example_value1>", "<example_value2>"],
      "query_hint": "<how_to_use_in_where_clause_of_SQL_query>"
    }}
  ]
}}

Use snake_case for all identifiers. You MUST infer the primary key based on uniqueness of values in the provided dataset. Use null if not applicable for any field. Output only the final JSON.

"""

metadata_info_prompt = """
Genreate the metadata for the following dataset:
{sample_data}
"""

add_info_prompt = """
In addition to the above, if you are provided with additional information about the dataset, your task is to incorporate that information into the metadata generation process. This may include:
- Additional context about the dataset
- Specific instructions or constraints that should be reflected in the metadata
- Any other relevant details that can enhance the understanding of the dataset

Below is the additional information you should consider:
{add_info}

"""

In [62]:
def metadata_generator(source_data_json: str, add_info_txt: str = None):
    """
    Generate metadata for one or more datasets using an LLM.
    This function uses the metadata_generator_prompt to create structured metadata.
    Accepts multiple JSON string representations of datasets.
    """

    # Combine all provided information into one string for the prompt
    user_prompt = metadata_info_prompt.format(sample_data=source_data_json)
    if add_info_txt:
        prompt += add_info_prompt.format(add_info=add_info_txt)

    # Input messages for LLM
    messages = [
            SystemMessage(content=metadata_generator_system_prompt),
            HumanMessage(content=user_prompt)
        ]
    
    # Call the LLM with the metadata generation prompt
    response = llm.invoke(messages)

    # Extract the content from AIMessage
    metadata_text = response.content if hasattr(response, 'content') else str(response)
    print("LLM raw response:", metadata_text)

    # Extract JSON object from the response using regex
    match = re.search(r'\{.*\}', metadata_text, re.DOTALL)
    if match:
        json_str = match.group(0)
        try:
            metadata_json = json.loads(json_str)
            print("Metadata generated successfully.")
        except Exception as e:
            print("Error parsing extracted JSON:", e)
            print("Extracted JSON was:\n", json_str)
            return None
    else:
        print("No JSON object found in LLM response. Unable to generate metadata")
        print("LLM response was:\n", metadata_text)
        return None

    return metadata_json


In [40]:
# Generate metadata for the aircraft dataset
aircraft_metadata = metadata_generator(aircraft_data_json)

LLM raw response: ```json
{
  "database": "aircraft_database",
  "table": "aircraft_specifications",
  "description": "A table containing specifications and details of various aircraft types.",
  "constraints": {
    "primary_key": ["aircraft_type"],
    "foreign_keys": []
  },
  "columns": [
    {
      "name": "aircraft_type",
      "type": "string",
      "description": "The name or type of the aircraft.",
      "examples": ["Airbus", "Boeing"],
      "query_hint": "WHERE aircraft_type = 'Airbus'"
    },
    {
      "name": "manufacturer",
      "type": "string",
      "description": "The company that manufactures the aircraft.",
      "examples": ["Airbus SE", "Boeing"],
      "query_hint": "WHERE manufacturer = 'Boeing'"
    },
    {
      "name": "typical_role",
      "type": "string",
      "description": "The typical role or usage of the aircraft.",
      "examples": ["Commercial Cargo", "Regional Commercial"],
      "query_hint": "WHERE typical_role = 'Commercial Cargo'"
    }

In [41]:
# Load generated aicraft metadata into MongoDB
load_metadata_json_str(aircraft_metadata)

Metadata uploaded successfully with ID: 687b479eb62030e1e2301368


In [49]:
# Generate metadata for the SG weather dataset with additional info
sgweather_metadata = metadata_generator(sgweather_data_json, weather_add_info)

LLM raw response: ```json
{
  "database": "weather_data",
  "table": "daily_weather",
  "description": "Contains daily weather observations including temperature, humidity, pressure, visibility, wind, precipitation, and events.",
  "constraints": {
    "primary_key": ["date"],
    "foreign_keys": []
  },
  "columns": [
    {
      "name": "date",
      "type": "date",
      "description": "The date of the weather observation in YYYY-MM-DD format.",
      "examples": ["2015-10-29", "2016-11-01"],
      "query_hint": "WHERE date = 'YYYY-MM-DD'"
    },
    {
      "name": "temp_high_f",
      "type": "integer",
      "description": "High temperature of the day in Fahrenheit.",
      "examples": ["81", "91"],
      "query_hint": "WHERE temp_high_f > 80"
    },
    {
      "name": "temp_avg_f",
      "type": "integer",
      "description": "Average temperature of the day in Fahrenheit.",
      "examples": ["66", "81"],
      "query_hint": "WHERE temp_avg_f BETWEEN 60 AND 70"
    },
    {
  

In [50]:
# Load generated SG weather metadata into MongoDB
load_metadata_json_str(sgweather_metadata)

Metadata uploaded successfully with ID: 687b493eb62030e1e2301369


### Build a Metadata Query Agent
The Metadata Query Agent will analyze the query to suggest the tables and columns to use

In [66]:
# Metadata generation system prompt template for generating metadata from a dataset.
# Escape curly braces for .format(), except for the data sample placeholder
metadata_query_system_prompt = """
You are a metadata query assistant. Your role is to help users understand the available data structures and guide them toward the right tables, columns, and relationships based on their queries.

You are provided with a metadata catalog (formatted as JSON) that includes databases, tables, columns, descriptions, relationships (primary/foreign keys), and query hints. Use this information to answer user queries in natural language.

### Instructions:
- Interpret the user query and identify the user's intent.
- Search the metadata for relevant tables and columns that could help answer the user’s question.
- If matching tables/columns exist, explain:
  - Which table(s) are relevant and why.
  - What useful columns are included in those tables.
  - What relationships exist between these tables (e.g., foreign keys).
  - Suggestions for how the user might combine or explore the data further.
- If the metadata doesn't include any relevant tables or fields, clearly inform the user that the requested data is not available.
- Be helpful and proactive — suggest potentially useful related tables when appropriate (e.g., weather, aircraft specs, etc.).
- Never make up table or column names that don't exist in the metadata.
- Be concise, clear, and informative.

### Metadata Catalog:
{metadata}

"""



In [68]:
# HARDCODED EXAMPLES. Need to pull from MongoDB and user query from frontend
metadata_from_MongoDB = """
{'database': 'airspace', 'collection': 'metadata_full', 'table': 'safety', 'description': 'Each record represents a single safety incident reported for aircraft operations, including failure parts, incident nature, precautionary actions, and contextual metadata such as date, location, and airline group/unit responsible.', 'constraints': {'primary_key': ['report_id'], 'foreign_keys': []}, 'columns': [{'name': 'date', 'type': 'DATE', 'description': 'Date of the incident in YYYY-MM-DD format. Range: 2020-01-01 to 2020-12-31.', 'examples': ['2020-01-15', '2020-06-30', '2020-11-05'], 'query_hint': 'Useful for filtering incidents by time period or aggregating counts over days, months, or quarters.'}, {'name': 'report', 'type': 'TEXT', 'description': 'A textual summary describing the incident details.', 'examples': ['ENGINE FAILURE DURING TAKEOFF, FUEL LEAK DETECTED', 'FUSELAGE CRACKED, EMERGENCY LIGHTS INOPERATIVE', 'ZONE 100 CORRODED, UNSCHED LANDING REQUIRED'], 'query_hint': "Use LIKE or full-text search for keywords (e.g., 'ENGINE', 'FUEL')."}, {'name': 'part_failure', 'type': 'VARCHAR', 'description': 'Specific part or system that failed during the incident.', 'examples': ['FUSELAGE CRACKED', 'ZONE 100 CORRODED', 'EMERGENCY LIGHTS INOPERATIVE'], 'query_hint': "Can be used to group or filter incidents by specific failure types (e.g., WHERE part_failure LIKE '%ZONE%')."}, {'name': 'occurence_nature_condition', 'type': 'VARCHAR', 'description': 'Describes the nature or condition of the incident occurrence, representing the type or symptoms observed.', 'examples': ['OTHER', 'WARNING INDICATION', 'NO TEST', 'SMOKE/FUMES/ODORS/SPARKS', 'FALSE WARNING', 'FLUID LOSS', 'FLT CONT AFFECTED'], 'query_hint': 'Useful for categorizing incidents by severity or symptomatology.'}, {'name': 'occurence_precautionary_procedures', 'type': 'VARCHAR', 'description': 'Records any precautionary or emergency procedures taken in response to the incident.', 'examples': ['UNSCHED LANDING', 'ABORTED TAKEOFF', 'RETURN TO BLOCK', 'EMER. DESCENT', 'ENGINE SHUTDOWN', 'ABORTED APPROACH', 'DEACTIVATE SYST/CIRCUITS'], 'query_hint': 'Useful to assess safety responses and actions triggered by incidents.'}, {'name': 'location', 'type': 'VARCHAR', 'description': 'The country where the incident took place.', 'examples': ['Singapore', 'Australia', 'France', 'Thailand', 'Brunei', 'United State'], 'query_hint': 'Examples showns are the only valid locations in the database. Do not use args other than those'}, {'name': 'airline_group', 'type': 'VARCHAR', 'description': 'The airline group or company responsible for the aircraft.', 'examples': ['SkyHigh Airways', 'AeroExpress', 'Pacific Aero', 'NimbusJet'], 'query_hint': 'Examples showns are the only valid airline group in the database. Do not use args other than those'}, {'name': 'unit', 'type': 'VARCHAR', 'description': 'Specific operational unit within the airline group accountable for the aircraft or incident. Each unit belongs uniquely to one group.', 'examples': ['SH Operations - East', 'SH Fleet A', 'AE Regional Hub 1', 'AE Long Haul Division', 'PA Crew Team Alpha', 'PA Engineering', 'NJ International Ops', 'NJ Domestic Base', 'SH Flight Ops HQ', 'AE Maintenance Unit', 'PA Ops Central', 'NJ Line Services'], 'query_hint': 'Examples showns are the only valid airline group in the database.  Do not use args other than those'}, {'name': 'aircraft_type', 'type': 'VARCHAR', 'description': 'the type of aircraft involved in the safety incident', 'examples': ['Airbus', 'Boeing', 'Hawker', 'Bombardier', 'Douglas', 'Fokker', 'Gulfstream', 'Hercules', 'Cessna', 'Convair', 'Embraer', 'Avro'], 'query_hint': 'Examples showns are the only valid aircraft type in the database.  Do not use args other than those'}]}
{'database': 'airspace', 'table': 'aircraft_specifications', 'description': 'A table containing specifications and details of various aircraft types.', 'constraints': {'primary_key': ['aircraft_type'], 'foreign_keys': []}, 'columns': [{'name': 'aircraft_type', 'type': 'string', 'description': 'The name or type of the aircraft.', 'examples': ['Airbus', 'Boeing'], 'query_hint': "WHERE aircraft_type = 'Airbus'"}, {'name': 'manufacturer', 'type': 'string', 'description': 'The company that manufactures the aircraft.', 'examples': ['Airbus SE', 'Boeing'], 'query_hint': "WHERE manufacturer = 'Boeing'"}, {'name': 'typical_role', 'type': 'string', 'description': 'The typical role or usage of the aircraft.', 'examples': ['Commercial Cargo', 'Regional Commercial'], 'query_hint': "WHERE typical_role = 'Commercial Cargo'"}, {'name': 'seating_capacity', 'type': 'string', 'description': 'The range of seating capacity available in the aircraft.', 'examples': ['100-850', '40-850'], 'query_hint': "WHERE seating_capacity = '100-850'"}, {'name': 'max_range_(km)', 'type': 'string', 'description': 'The maximum range of the aircraft in kilometers.', 'examples': ['4000-15000', '3000-15000'], 'query_hint': "WHERE max_range_(km) = '4000-15000'"}, {'name': 'first_flight_year', 'type': 'integer', 'description': 'The year when the aircraft had its first flight.', 'examples': [1972, 1958], 'query_hint': 'WHERE first_flight_year = 1972'}, {'name': 'aircraft_class', 'type': 'string', 'description': 'The classification of the aircraft based on its design and usage.', 'examples': ['Wide-body, Narrow-body', 'Regional, Narrow-body'], 'query_hint': "WHERE aircraft_class = 'Wide-body, Narrow-body'"}, {'name': 'cargo_capacity_(kg)', 'type': 'string', 'description': 'The range of cargo capacity in kilograms that the aircraft can carry.', 'examples': ['10000-80000', '10000-60000'], 'query_hint': "WHERE cargo_capacity_(kg) = '10000-80000'"}, {'name': 'notable_models', 'type': 'string', 'description': 'Notable models of the aircraft type.', 'examples': ['A320, A330, A380', '737, 747, 787'], 'query_hint': "WHERE notable_models LIKE '%A320%'"}]}
{'database': 'airspace', 'table': 'daily_weather', 'description': 'Contains daily weather observations including temperature, humidity, pressure, visibility, wind, precipitation, and events.', 'constraints': {'primary_key': ['date'], 'foreign_keys': []}, 'columns': [{'name': 'date', 'type': 'date', 'description': 'The date of the weather observation in YYYY-MM-DD format.', 'examples': ['2015-10-29', '2016-11-01'], 'query_hint': "WHERE date = 'YYYY-MM-DD'"}, {'name': 'temp_high_f', 'type': 'integer', 'description': 'High temperature of the day in Fahrenheit.', 'examples': ['81', '91'], 'query_hint': 'WHERE temp_high_f > 80'}, {'name': 'temp_avg_f', 'type': 'integer', 'description': 'Average temperature of the day in Fahrenheit.', 'examples': ['66', '81'], 'query_hint': 'WHERE temp_avg_f BETWEEN 60 AND 70'}, {'name': 'temp_low_f', 'type': 'integer', 'description': 'Low temperature of the day in Fahrenheit.', 'examples': ['51', '71'], 'query_hint': 'WHERE temp_low_f < 50'}, {'name': 'dew_point_high_f', 'type': 'integer', 'description': 'High dew point of the day in Fahrenheit.', 'examples': ['64', '73'], 'query_hint': 'WHERE dew_point_high_f > 70'}, {'name': 'dew_point_avg_f', 'type': 'integer', 'description': 'Average dew point of the day in Fahrenheit.', 'examples': ['54', '71'], 'query_hint': 'WHERE dew_point_avg_f BETWEEN 50 AND 60'}, {'name': 'dew_point_low_f', 'type': 'integer', 'description': 'Low dew point of the day in Fahrenheit.', 'examples': ['49', '64'], 'query_hint': 'WHERE dew_point_low_f < 50'}, {'name': 'humidity_high_percent', 'type': 'integer', 'description': 'High humidity of the day as a percentage.', 'examples': ['96', '100'], 'query_hint': 'WHERE humidity_high_percent > 90'}, {'name': 'humidity_avg_percent', 'type': 'integer', 'description': 'Average humidity of the day as a percentage.', 'examples': ['66', '72'], 'query_hint': 'WHERE humidity_avg_percent BETWEEN 60 AND 70'}, {'name': 'humidity_low_percent', 'type': 'integer', 'description': 'Low humidity of the day as a percentage.', 'examples': ['35', '44'], 'query_hint': 'WHERE humidity_low_percent < 40'}, {'name': 'sea_level_pressure_high_inches', 'type': 'float', 'description': 'High sea level pressure of the day in inches.', 'examples': ['29.97', '30.04'], 'query_hint': 'WHERE sea_level_pressure_high_inches > 30'}, {'name': 'sea_level_pressure_avg_inches', 'type': 'float', 'description': 'Average sea level pressure of the day in inches.', 'examples': ['29.89', '29.98'], 'query_hint': 'WHERE sea_level_pressure_avg_inches BETWEEN 29.8 AND 30'}, {'name': 'sea_level_pressure_low_inches', 'type': 'float', 'description': 'Low sea level pressure of the day in inches.', 'examples': ['29.83', '29.9'], 'query_hint': 'WHERE sea_level_pressure_low_inches < 29.9'}, {'name': 'visibility_high_miles', 'type': 'integer', 'description': 'High visibility of the day in miles.', 'examples': ['10', '10'], 'query_hint': 'WHERE visibility_high_miles = 10'}, {'name': 'visibility_avg_miles', 'type': 'integer', 'description': 'Average visibility of the day in miles.', 'examples': ['10', '9'], 'query_hint': 'WHERE visibility_avg_miles < 10'}, {'name': 'visibility_low_miles', 'type': 'integer', 'description': 'Low visibility of the day in miles.', 'examples': ['10', '1'], 'query_hint': 'WHERE visibility_low_miles < 5'}, {'name': 'wind_high_mph', 'type': 'integer', 'description': 'High wind speed of the day in miles per hour.', 'examples': ['12', '15'], 'query_hint': 'WHERE wind_high_mph > 10'}, {'name': 'wind_avg_mph', 'type': 'integer', 'description': 'Average wind speed of the day in miles per hour.', 'examples': ['4', '5'], 'query_hint': 'WHERE wind_avg_mph BETWEEN 3 AND 6'}, {'name': 'wind_gust_mph', 'type': 'integer', 'description': 'Highest wind speed gust of the day in miles per hour.', 'examples': ['18', '22'], 'query_hint': 'WHERE wind_gust_mph > 20'}, {'name': 'precipitation_sum_inches', 'type': 'float', 'description': 'Total precipitation of the day in inches.', 'examples': ['0', '0.33'], 'query_hint': 'WHERE precipitation_sum_inches > 0'}, {'name': 'events', 'type': 'string', 'description': 'Adverse weather events recorded for the day. Empty if none.', 'examples': [' ', 'Rain , Thunderstorm'], 'query_hint': "WHERE events LIKE '%Rain%'"}]}
"""

user_query = "I want to know the incidents for different aircraft over the years"


In [69]:
def metadata_query(user_query: str, metadata_json: str):
    """
    Query the metadata catalog to find relevant tables and columns based on a user query.
    
    Args:
        user_query (str): The user's query asking for specific data or insights.
        metadata_json (str): JSON string containing the metadata catalog.
    Returns:
        str: Response containing relevant tables, columns, and relationships.
    """
    
    # Prepare the metadata for the prompt
    metadata = json.dumps(metadata_json, indent=2)
    
    # Prepare the user query
    user_prompt = user_query
    
    # Create the system prompt
    system_prompt = metadata_query_system_prompt.format(metadata=metadata)
    
    # Input messages for LLM
    messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
    
    # Call the LLM with the metadata query prompt
    response = llm.invoke(messages)

    # Extract the content from AIMessage
    return response.content if hasattr(response, 'content') else str(response)

In [65]:
docs = metadata_full.find({}, {"_id": 0}) 
for doc in docs:
    print(doc)

{'database': 'airspace', 'collection': 'metadata_full', 'table': 'safety', 'description': 'Each record represents a single safety incident reported for aircraft operations, including failure parts, incident nature, precautionary actions, and contextual metadata such as date, location, and airline group/unit responsible.', 'constraints': {'primary_key': ['report_id'], 'foreign_keys': []}, 'columns': [{'name': 'date', 'type': 'DATE', 'description': 'Date of the incident in YYYY-MM-DD format. Range: 2020-01-01 to 2020-12-31.', 'examples': ['2020-01-15', '2020-06-30', '2020-11-05'], 'query_hint': 'Useful for filtering incidents by time period or aggregating counts over days, months, or quarters.'}, {'name': 'report', 'type': 'TEXT', 'description': 'A textual summary describing the incident details.', 'examples': ['ENGINE FAILURE DURING TAKEOFF, FUEL LEAK DETECTED', 'FUSELAGE CRACKED, EMERGENCY LIGHTS INOPERATIVE', 'ZONE 100 CORRODED, UNSCHED LANDING REQUIRED'], 'query_hint': "Use LIKE or f

In [70]:
test = metadata_query(user_query, metadata_from_MongoDB)

In [73]:
sentences = test.split('. ')
for sentence in sentences:
    print(sentence.strip() + '.')

To explore incidents for different aircraft over the years, you can refer to the `safety` table in the metadata catalog.
This table contains records of safety incidents reported for aircraft operations.
Here are the relevant details:

- **Table**: `safety`
  - **Relevant Columns**:
    - `date`: This column provides the date of the incident, which you can use to filter or aggregate incidents over specific time periods (e.g., by year).
    - `aircraft_type`: This column indicates the type of aircraft involved in the incident, allowing you to group or filter incidents by aircraft type.
    - `report`: A textual summary describing the incident details.
    - `part_failure`: Specific part or system that failed during the incident.
    - `occurence_nature_condition`: Describes the nature or condition of the incident occurrence.
    - `occurence_precautionary_procedures`: Records any precautionary or emergency procedures taken in response to the incident.

You can query this table to get inc